In [2]:
#Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#Acceder al archivo CSV
dataframe = pd.read_csv('Maternal Health Risk Data Set.csv')

In [6]:
#Definimos los valores para entrenar al algoritmo
columnasdf = dataframe.columns
vardep = columnasdf[len(columnasdf)-1]
categorias = {"low risk": 1, "mid risk": 2, "high risk": 3}

x = dataframe.drop(vardep, axis=1)
y = dataframe[vardep]
y = y.map(categorias)

In [10]:
#Entrenar el modelo con los valores óptimos hallados anteriormente
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=30)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_esc = scaler.fit_transform(x_train)
x_test_esc = scaler.transform(x_test)

from sklearn.ensemble import RandomForestClassifier
modeloRF = RandomForestClassifier(n_estimators=56, random_state=42)
modeloRF.fit(x_train_esc, y_train)

RandomForestClassifier(n_estimators=56, random_state=42)

In [18]:
#Librerías para la conección entre el formulario HTML y el archivo Jupiter
from flask import Flask, render_template, request
import os
from io import BytesIO

In [20]:
#Nombre del módulo actual y su dirección
template_dir = os.path.abspath(os.path.join(os.getcwd(), '../Formulario/templates'))
app = Flask(__name__, template_folder=template_dir)

In [22]:
#Ruta que permite la comunicación con el formulario y métodos que le son permitidos
@app.route("/", methods=["GET", "POST"])
#Definimos la función de respuesta
def formulario():
    resultado = None
    try:
        accion=request.form.get("envio")
        if accion == "sin_pdf":
            #Datos recibidos del formulario
            datos_formulario = []
            for i in x.columns:
                if i == "BS" or i == "BodyTemp":
                    datos_formulario.append(float(request.form[i]))
                else:
                    datos_formulario.append(int(request.form[i]))

            #Creación de dataframe de datos y escalado
            nuevos_datos = pd.DataFrame([datos_formulario], columns=x.columns)
            nuevos_datos_esc = scaler.transform(nuevos_datos)
            
            #Verifica si los datos recibidos son coherentes con las indicaciones de la base de datos
            if (nuevos_datos > 0).all().all():
                #Respuesta del algoritmo
                prediccion = modeloRF.predict(nuevos_datos_esc)
                prob_prediccion = modeloRF.predict_proba(nuevos_datos_esc)
                
                #Generando respuesta según los resultados de la predicción
                match int(prediccion[0]):
                    case 1:
                        probabilidad = round(float(prob_prediccion[0][0])*100,2)
                        resultado = f"El resultado es de bajo riesgo con una probabilidad de {probabilidad}%."
                    case 2:
                        probabilidad = round(float(prob_prediccion[0][1])*100,2)
                        resultado = f"El resultado es de riesgo medio con una probabilidad de {probabilidad}%."
                    case 3:
                        probabilidad = round(float(prob_prediccion[0][2])*100,2)
                        resultado = f"El resultado es de alto riesgo con una probabilidad de {probabilidad}%."
                print("Respuesta exitosa")
            else:
                resultado = "Datos inválidos, verifique si la información es correcta"
                print("Consulta inválida")
    
    except ValueError:
        resultado = "Hubo un error X_X, verifique sus datos"
        print("Error detectado")

    #Se busca el archivo formulario.html en la carpeta templates y se envía de respuesta
    return render_template("formulario.html", resultado=resultado)

In [24]:
#Ejecución del archivo
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [25/May/2025 22:05:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2025 22:11:56] "POST / HTTP/1.1" 200 -


Respuesta exitosa


127.0.0.1 - - [25/May/2025 22:13:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2025 22:15:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2025 22:17:08] "POST / HTTP/1.1" 200 -


Respuesta exitosa


127.0.0.1 - - [25/May/2025 22:25:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2025 22:26:48] "POST / HTTP/1.1" 200 -


Respuesta exitosa
